In [76]:
# Instalar a biblioteca do cliente do Google e a biblioteca para autenticação
#!pip install --upgrade google-api-python-client google-auth google-auth-httplib2 google-auth-oauthlib

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
import os
from google.colab import auth
from google.oauth2 import service_account
from googleapiclient.discovery import build

import pandas as pd
import numpy as np

from google.colab import drive

# Passo 1: Montar o Google Drive (necessário em cada novo notebook)
# Uma janela de autenticação aparecerá para você permitir o acesso.
drive.mount('/content/drive')

# Passo 2: Definir o caminho completo do arquivo que você quer carregar
# ATENÇÃO: Use o mesmo caminho que você usou para salvar o arquivo.
caminho_do_arquivo = '/content/drive/MyDrive/MBA/TCC/Materiais_e_Metodos/Dados/dados_limpos.csv'

# Passo 3: Carregar o arquivo CSV em um DataFrame do Pandas
# Usamos um bloco try...except para dar uma mensagem de erro clara se o arquivo não for encontrado.
try:
    df = pd.read_csv(caminho_do_arquivo)
    print("Arquivo carregado com sucesso!")

    # Opcional: Mostra as 5 primeiras linhas para confirmar que os dados estão corretos
    print("Amostra dos dados carregados:")
    pd.set_option('display.max_columns', None)
    print(df.head())

except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado no caminho especificado.")
    print(f"Verifique se o caminho '{caminho_do_arquivo}' está correto.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Arquivo carregado com sucesso!
Amostra dos dados carregados:
             timestamp  idade_ate_25      idade_26_35 idade_36_45  \
0  29/05/2025 14:25:27           NaN              NaN         NaN   
1  30/05/2025 21:43:56           NaN    Pós-graduação         NaN   
2  31/05/2025 12:03:02           NaN  Ensino Superior         NaN   
3  31/05/2025 17:40:18           NaN  Ensino Superior         NaN   
4  02/06/2025 10:09:19  Ensino Médio              NaN         NaN   

  idade_46_ou_mais        preta        parda  indigena amarela branca  \
0    Pós-graduação          NaN  Mulher(cis)       NaN     NaN    NaN   
1              NaN  Mulher(cis)          NaN       NaN     NaN    NaN   
2              NaN  Mulher(cis)          NaN       NaN     NaN    NaN   
3              NaN          NaN  Mulher(cis)       NaN     NaN    NaN   
4              NaN  Mulher(cis

In [79]:
# Mostrar o número de respostas e de perguntas
print(f"Total de respostas (participantes): {df.shape[0]}")
print(f"Total de perguntas (colunas): {df.shape[1]}")
print("\n")

Total de respostas (participantes): 41
Total de perguntas (colunas): 38




In [80]:
# Analisar a frequência das respostas para o perfil demográfico e profissional
# Usando os nomes originais das colunas, como estão no seu arquivo bruto.
print("### Análise de Frequência - Perfil da Amostra ###")
print("-" * 40)

### Análise de Frequência - Perfil da Amostra ###
----------------------------------------


In [81]:
# 3.3 Tempo de Experiência

print("3. Opinião referente as Vagas Afirmativas:")
if 'opiniao_vagas_afirmativas' in df.columns:
    # 1. Padroniza todos os tipos de "respostas vazias" (strings vazias ou com espaços) para NaN
    opinioes_limpas = df['opiniao_vagas_afirmativas'].replace(r'^\s*$', np.nan, regex=True)
    opiniao_vagas_afirmativas_counts = df['opiniao_vagas_afirmativas'].value_counts(normalize=True).mul(100).round(1)

    # 2. Remove todas as linhas que são NaN
    opinioes_validas = opinioes_limpas.dropna()

    # 3. Verifica se ainda restam respostas após a limpeza
    if not opinioes_validas.empty:
        # Imprime a lista de respostas válidas, sem o número do índice
        print(opinioes_validas.to_string(index=False))
        #print(opiniao_vagas_afirmativas_counts.to_string())
        #print(df['opiniao_vagas_afirmativas'].to_string())
    else:
        print("Nenhuma resposta preenchida foi encontrada para esta pergunta.")

else:
    print("A coluna 'opiniao_vagas_afirmativas' não foi encontrada no DataFrame.")

print("\n")

3. Opinião referente as Vagas Afirmativas:
Essenciais para melhorar a diversidade no traba...
As vagas afirmativas são de extrema importância...
De extrema importância, desde que as vagas não ...
As vagas afirmativas têm um papel de 'porta de ...
Acredito que as vagas afirmativas ajudam a dar ...
Dado o contexto social que vivemos no Brasil e ...
Vejo pontos positivos e negativos no sentido de...
A área de tecnologia não é um ambiente inclusiv...
Entrei no GB graças a um programa que ajudava p...
Na minha visão, as vagas afirmativas são fundam...
acredito que são essenciais , mas não basta só ...
Muito importante desde que sejam realizadas com...
Acredito que elas devem existir pois há uma con...
Acho muito importante ter vagas afirmativas, ma...
Acho super importante. Num país com tamanha des...
São essenciais se a empresa tenta ativamente bu...
É importante para ter oportunidades para grupos...




**Análise de Sentimento**
A análise de sentimento é uma técnica de Processamento de Linguagem Natural (PLN) que nos permite identificar e extrair opiniões e emoções de textos. No seu TCC, ela será extremamente valiosa para quantificar a tonalidade das experiências que as mulheres negras relataram nas perguntas abertas.

Por que isso é importante? Em vez de apenas ler e interpretar subjetivamente as respostas, podemos classificar cada uma delas como positiva, negativa ou neutra. Isso nos dará uma métrica poderosa para, por exemplo, correlacionar o sentimento geral das participantes com suas senioridades, tempo de experiência ou se elas ingressaram por vaga afirmativa.

Vamos aplicar essa análise em duas das colunas mais ricas do seu questionário:

desafios_carreira

percepcao_inclusao

In [82]:
# 1. Instalação da biblioteca
!pip install transformers

# 2. Importação das bibliotecas necessárias
from transformers import pipeline, set_seed

set_seed(42)
# 3. Carregar o modelo de análise de sentimento para português
# O modelo "lxyuan/distilbert-base-multilingual-cased-sentiments-student" é uma boa opção
# por ser leve e suportar múltiplos idiomas, incluindo o nosso.
sentiment_pipeline = pipeline("sentiment-analysis", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

print("✅ Modelo de análise de sentimento carregado com sucesso.")

Device set to use cpu


✅ Modelo de análise de sentimento carregado com sucesso.


novas colunas no DataFrame para armazenar o resultado (o sentimento e a confiança da previsão)

In [83]:
# 4. Função para aplicar a análise e extrair o resultado
def analisar_sentimento(texto):
    # Verifica se o texto não é nulo/vazio
    if texto and isinstance(texto, str):
        resultado = sentiment_pipeline(texto)
        return resultado[0]['label'] # Retorna 'positive', 'negative' ou 'neutral'
    return None # Retorna Nulo se o campo estiver vazio

# 5. Aplicar a função nas colunas de interesse
# Cuidado: Isso pode demorar alguns minutos dependendo da quantidade de respostas
# 1. Crie uma lista com os nomes das colunas que você deseja analisar
colunas_para_analise = [
    'enfrentou_barreiras',
    'impacto_vaga_afirmativa',
    'sentimento_valorizacao',
    'oportunidades_desenvolvimento',
    'opiniao_vagas_afirmativas',
    'fatores_permanencia'
]

print("Iniciando a análise de sentimento...")

# 2. Crie um loop que itera sobre a lista de colunas
for coluna in colunas_para_analise:
    # Gera dinamicamente o nome da nova coluna para os resultados
    nova_coluna = f'sentimento_{coluna}'

    print(f"Processando coluna: '{coluna}'...")

    # Aplica a função à coluna atual e armazena o resultado na nova coluna
    df[nova_coluna] = df[coluna].apply(analisar_sentimento)

print("\n✅ Análise de sentimento concluída para todas as colunas!")



Iniciando a análise de sentimento...
Processando coluna: 'enfrentou_barreiras'...
Processando coluna: 'impacto_vaga_afirmativa'...
Processando coluna: 'sentimento_valorizacao'...
Processando coluna: 'oportunidades_desenvolvimento'...
Processando coluna: 'opiniao_vagas_afirmativas'...
Processando coluna: 'fatores_permanencia'...

✅ Análise de sentimento concluída para todas as colunas!


In [84]:
# 1. Crie um dicionário para mapear os nomes das colunas aos títulos desejados
colunas_e_titulos = {
    'sentimento_enfrentou_barreiras': 'Desafios na Carreira',
    'sentimento_impacto_vaga_afirmativa': 'Impacto Percebido da Vaga Afirmativa',
    'sentimento_valorizacao': 'Sentimento de Valorização Profissional',
    'sentimento_oportunidades_desenvolvimento': 'Percepção sobre Oportunidades de Desenvolvimento',
    'sentimento_opiniao_vagas_afirmativas': 'Opinião Geral sobre Vagas Afirmativas',
    'sentimento_fatores_permanencia': 'Fatores Relevantes para a Permanência'
}

print("\n--- Análise da Distribuição de Sentimentos ---\n")

# 2. Faça um loop 'for' que itera sobre os itens do dicionário
for coluna, titulo in colunas_e_titulos.items():
    print(f"Distribuição de sentimentos sobre: '{titulo}'")

    # Calcula e exibe a distribuição percentual para a coluna atual
    distribuicao = df[coluna].value_counts(normalize=True).mul(100).round(1)
    print(distribuicao)
    print("-" * 50) # Adiciona uma linha para separar as análises



--- Análise da Distribuição de Sentimentos ---

Distribuição de sentimentos sobre: 'Desafios na Carreira'
sentimento_enfrentou_barreiras
positive    100.0
Name: proportion, dtype: float64
--------------------------------------------------
Distribuição de sentimentos sobre: 'Impacto Percebido da Vaga Afirmativa'
sentimento_impacto_vaga_afirmativa
positive    100.0
Name: proportion, dtype: float64
--------------------------------------------------
Distribuição de sentimentos sobre: 'Sentimento de Valorização Profissional'
sentimento_valorizacao
Concordo               46.3
Concordo Totalmente    43.9
Neutro                  9.8
Name: proportion, dtype: float64
--------------------------------------------------
Distribuição de sentimentos sobre: 'Percepção sobre Oportunidades de Desenvolvimento'
sentimento_oportunidades_desenvolvimento
positive    100.0
Name: proportion, dtype: float64
--------------------------------------------------
Distribuição de sentimentos sobre: 'Opinião Geral sob

## **Cruzamento de Todas as Colunas**

In [ ]:
from itertools import combinations # Importa a função para criar combinações

# --- PRÉ-REQUISITO ---
# Assumimos que o seu DataFrame 'df' já foi carregado do seu arquivo CSV do Drive.
# Todo o código abaixo vai operar NESSE 'df', sem nunca substituí-lo.


# --- Sua função de análise (não precisa mudar nada aqui) ---
def analisar_cruzamento(dataframe, coluna_perfil, coluna_dependente):
    """Gera e exibe tabelas de contagem e percentual para duas colunas."""
    titulo_perfil = coluna_perfil.replace('_', ' ').capitalize()
    titulo_dependente = coluna_dependente.replace('_', ' ').capitalize()
    print(f"--- Análise Cruzada: {titulo_perfil} vs. {titulo_dependente} ---")
    tabela_contagem = pd.crosstab(index=dataframe[coluna_perfil], columns=dataframe[coluna_dependente])
    print("\nTabela de Contagem Absoluta:\n")
    print(tabela_contagem)
    tabela_percentual = pd.crosstab(index=dataframe[coluna_perfil], columns=dataframe[coluna_dependente], normalize='index').mul(100).round(1)
    print("\n\nTabela de Percentual (por linha):\n")
    print(tabela_percentual)
    print("\n" + "="*70 + "\n")

# --- INÍCIO DA SOLUÇÃO SEGURA E AUTOMATIZADA ---

# 1. (AUTOMÁTICO) Obtém a lista de nomes de todas as colunas do SEU DataFrame.
lista_de_colunas = df.columns.tolist()

# 2. (AUTOMÁTICO) Gera todas as combinações únicas de pares (2 a 2) a partir da lista de colunas.
lista_de_cruzamentos = list(combinations(lista_de_colunas, 2))

# Opcional: Imprime a lista gerada para você conferir
print("Cruzamentos que serão gerados automaticamente a partir das colunas do DataFrame:")
print(lista_de_cruzamentos)
print("-" * 70)


# 3. O loop executa a análise no seu 'df' original.
for perfil, dependente in lista_de_cruzamentos:
    analisar_cruzamento(df,
                        coluna_perfil=perfil,
                        coluna_dependente=dependente)

# **Cruzamento com colunas específicas**

In [102]:
# Escolha das variáveis para cruzar
# Variável de Perfil (independente): 'senioridade'
# Variável de Sentimento (dependente): 'sentimento_opiniao_vagas_afirmativas'
data = {
    'senioridade': np.random.choice(['Júnior', 'Pleno', 'Sênior'], 100),
    'sentimento_opiniao_vagas_afirmativas': np.random.choice(['Positivo', 'Neutro', 'Negativo'], 100, p=[0.6, 0.2, 0.2]),
    'tempo_carreira_anos': np.random.choice(['0-2 anos', '3-5 anos', '6+ anos'], 100)
}

# 2. Ttabela de contagem (números absolutos)
# Mostra quantos respondentes de cada senioridade deram cada tipo de resposta.
tabela_contagem = pd.crosstab(index=data['senioridade'], columns=data['sentimento_opiniao_vagas_afirmativas'])
tabela_contagem1 = pd.crosstab(index=data['tempo_carreira_anos'], columns=data['sentimento_opiniao_vagas_afirmativas'])
tabela_contagem2 = pd.crosstab(index=data['senioridade'], columns=data['tempo_carreira_anos'])

print("\nTabela de Contagem Absoluta:\n")
print("--- Análise Cruzada: Senioridade vs. Opinião sobre Vagas Afirmativas ---")
print(tabela_contagem)
print("--- Análise Cruzada: tempo_carreira_anos vs. Opinião sobre Vagas Afirmativas ---")
print(tabela_contagem1)
print("--- Análise Cruzada: Senioridade vs. tempo_carreira_anos ---")
print(tabela_contagem2)


# 3. Tabela de percentuais (mais poderosa para comparação)
# O argumento normalize='index' calcula a porcentagem ao longo das linhas.
# Isso nos ajuda a responder: "Dentro do grupo 'Sênior', qual a % de sentimentos positivos?"
tabela_percentual = pd.crosstab(index=data['senioridade'], columns=data['sentimento_opiniao_vagas_afirmativas'], normalize='index').mul(100).round(1)

print("\n\nTabela de Percentual (por linha/senioridade):\n")
print(tabela_percentual)


Tabela de Contagem Absoluta:

--- Análise Cruzada: Senioridade vs. Opinião sobre Vagas Afirmativas ---
col_0   Negativo  Neutro  Positivo
row_0                             
Júnior         5       9        29
Pleno          7       7        16
Sênior         8       1        18
--- Análise Cruzada: tempo_carreira_anos vs. Opinião sobre Vagas Afirmativas ---
col_0     Negativo  Neutro  Positivo
row_0                               
0-2 anos         7       6        18
3-5 anos         8       5        25
6+ anos          5       6        20
--- Análise Cruzada: Senioridade vs. tempo_carreira_anos ---
col_0   0-2 anos  3-5 anos  6+ anos
row_0                              
Júnior        15        14       14
Pleno          7        13       10
Sênior         9        11        7


Tabela de Percentual (por linha/senioridade):

col_0   Negativo  Neutro  Positivo
row_0                             
Júnior      11.6    20.9      67.4
Pleno       23.3    23.3      53.3
Sênior      29.6     3.7 

Realizar uma análise de sentimento nos comentários
Subtask:
Utilizar uma biblioteca ou abordagem para realizar a análise de sentimento nos comentários abertos e possivelmente em outras colunas de texto.

Reasoning: Selecionar as colunas de texto relevantes e realizar a análise de sentimento utilizando a biblioteca nltk.sentiment.SentimentIntensityAnalyzer, armazenando os resultados em novas colunas no DataFrame.

In [120]:
import nltk

# Baixar o lexicon VADER
nltk.download('vader_lexicon')

# Inicializar o analisador de sentimento VADER
# VADER é treinado para inglês e pode não ser ideal para português sem adaptação.
# Para uma análise mais robusta em português, seria necessária uma ferramenta ou modelo treinado especificamente para o idioma.
# Usaremos VADER aqui para demonstrar a abordagem, mas os resultados devem ser interpretados com cautela.
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Selecionar colunas de texto para análise de sentimento
sentiment_columns = ['opiniao_vagas_afirmativas']

# Realizar análise de sentimento em cada coluna
for col in sentiment_columns:
    # Preencher valores ausentes com string vazia
    df[col] = df[col].fillna('')

    # Aplicar análise de sentimento e criar novas colunas
    df[f'{col}_neg'] = df[col].apply(lambda text: analyzer.polarity_scores(text)['neg'])
    df[f'{col}_neu'] = df[col].apply(lambda text: analyzer.polarity_scores(text)['neu'])
    df[f'{col}_pos'] = df[col].apply(lambda text: analyzer.polarity_scores(text)['pos'])
    df[f'{col}_compound'] = df[col].apply(lambda text: analyzer.polarity_scores(text)['compound'])

# Exibir as primeiras linhas do DataFrame com as novas colunas de sentimento
print("DataFrame com resultados da análise de sentimento:")
display(df[['opiniao_vagas_afirmativas', 'opiniao_vagas_afirmativas_compound']].head())

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


DataFrame com resultados da análise de sentimento:


,opiniao_vagas_afirmativas,opiniao_vagas_afirmativas_compound
0,,0.000
1,Essenciais para melhorar a diversidade no trab...,-0.296
2,,0.000
3,,0.000
4,As vagas afirmativas são de extrema importânci...,0.000


In [121]:
# Definir limites para categorizar o sentimento com base na pontuação composta
# Estes limites são sugestões comuns, mas podem ser ajustados conforme necessário
def categorize_sentiment(score):
    if score >= 0.05:
        return 'Positivo'
    elif score <= -0.05:
        return 'Negativo'
    else:
        return 'Neutro'

# Aplicar a função para criar uma nova coluna de categoria de sentimento
df['opiniao_vagas_afirmativas_sentimento'] = df['opiniao_vagas_afirmativas_compound'].apply(categorize_sentiment)

# Contar a frequência de cada categoria de sentimento
sentiment_counts = df['opiniao_vagas_afirmativas_sentimento'].value_counts()

print("\nContagem de respostas por categoria de sentimento:")
display(sentiment_counts)



Contagem de respostas por categoria de sentimento:


,count
opiniao_vagas_afirmativas_sentimento,
Neutro,33
Negativo,7
Positivo,1


In [124]:
# Exemplo: Se você tem uma pasta chamada "TCC" e dentro dela uma subpasta "Dados"
caminho_da_pasta = '/content/drive/MyDrive/MBA/TCC/Materiais_e_Metodos/Dados/'
nome_do_arquivo = 'dados_com_sentimento.csv'
caminho_completo = caminho_da_pasta + nome_do_arquivo

# Supondo que o seu DataFrame se chame 'df'
# (Crie um DataFrame de exemplo se estiver testando)
# df = pd.DataFrame({'coluna1': [1, 2], 'coluna2': ['A', 'B']})

# Passo 3: Salvar o DataFrame no caminho especificado
# O argumento 'index=False' é importante para não salvar o índice do DataFrame no CSV.
df.to_csv(caminho_completo, index=False)

print(f"Arquivo '{nome_do_arquivo}' salvo com sucesso em:")
print(caminho_da_pasta)

Arquivo 'dados_com_sentimento.csv' salvo com sucesso em:
/content/drive/MyDrive/MBA/TCC/Materiais_e_Metodos/Dados/
